In [1]:
#loading all the data set 
assess= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\assessments.csv ", na.strings = c("","NA"))
courses= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\courses.csv ", na.strings = c("","NA"))
std_assess= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\studentAssessment.csv ", na.strings = c("","NA"))
std_info= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\studentInfo.csv ", na.strings = c("","NA"))
std_registration= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\studentRegistration.csv ", na.strings = c("","NA"))
std_vle= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\studentVle.csv ", na.strings = c("","NA"))
vle= read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\vle.csv ", na.strings = c("","NA"))

https://stackoverflow.com/questions/1299871/how-to-join-merge-data-frames-inner-outer-left-right

library(dplyr)
inner_dplyr <- inner_join(flights, weather, by = mergeCols)
left_dplyr  <- left_join(flights,  weather, by = mergeCols)
right_dplyr <- right_join(flights, weather, by = mergeCols)
full_dplyr  <- full_join(flights,  weather, by = mergeCols)

inner_dplyr <- flights %>% inner_join(weather, by = mergeCols)
left_dplyr  <- flights %>% left_join(weather,  by = mergeCols)
right_dplyr <- flights %>% right_join(weather, by = mergeCols)
full_dplyr  <- flights %>% full_join(weather,  by = mergeCols)

In [2]:
library(dplyr)
library(dummies)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

dummies-1.5.6 provided by Decision Patterns



In [3]:
#NA value here means that student didnt drop the class
head(std_registration)

#replacing those NA values with "stayed"
std_registration[is.na(std_registration$date_unregistration),"date_unregistration"] = "Completed"

code_module,code_presentation,id_student,date_registration,date_unregistration
AAA,2013J,11391,-159,NA
AAA,2013J,28400,-53,NA
AAA,2013J,30268,-92,12
AAA,2013J,31604,-52,NA
AAA,2013J,32885,-176,NA
AAA,2013J,38053,-110,NA


> ### Student with no unregistered were assumed to be the ones who didnt drop the class. Replaced those NA values with "Stayed" 

In [4]:
# There are instances with missing date_registration
# We can replace them by grouping the data by code_presentaion and taking the 
# mean for the know days and replace on the missing places 


In [5]:
#Joining table student registration with student Information by id_student
std_join_info_reg = std_registration %>% inner_join(std_info, by = "id_student") 

#deleting duplicate code modeule and code presentaion columns
std_join_info_reg = std_join_info_reg[,-which(names(std_join_info_reg) %in% c("code_presentation.y","code_module.y"))]

> ## If you do left join instead of inner join you realise that there was data of student registering certain courses but no associate student info with no information on their grades in that particular course.

In [6]:
length(unique(std_info$id_student))

[1] 23536

In [7]:
length(unique(std_registration$id_student))

[1] 28785

In [8]:
nrow(std_join_info_reg)

[1] 32734

# Preparing Vle join table:
## Joining student vle with vle by id_site

In [9]:
#joining vle student to vle data
vle_join = std_vle %>% left_join(vle, by = "id_site") 

In [10]:
#Checking NA values in the join
nrow(vle_join[!complete.cases(vle_join),])

[1] 9269544

In [11]:
nrow(vle_join)

[1] 10655280

### vle_join has 9269544 NA values for week_from

> ### Deleting week_from and week_to as more than 90% of them are empty

In [12]:
# dropping week_from and week_to
vle_join = vle_join[,-which(names(vle_join) %in%  c("week_from","week_to"))]

In [13]:
# no NA values in vle table check 
nrow(vle_join[!complete.cases(vle_join),])
#deleting duplicate code modeule and code presentaion columns
vle_join = vle_join[,-which(names(vle_join) %in% c("code_presentation.y","code_module.y"))]

[1] 0

# Joining student assessment with assessment by id_assessment:


In [14]:
#JOINING
assess_join = std_assess %>% left_join(assess, by = "id_assessment") 

In [15]:
nrow(assess_join)

[1] 173912

# Working on joining the joined 3 dataframes

In [16]:
tom = dummy(vle_join$activity_type,sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"

> ### Created dummy variables for activity_type 

In [17]:
vle_join_clean_withdummy = cbind(vle_join,tom)
vle_join_clean_withdummy = vle_join_clean_withdummy[,-which(names(vle_join_clean_withdummy) %in% c("activity_type","code_presentation.y","code_module.y","id_site"))]

In [18]:
length(unique(vle_join_clean_withdummy$date))
names(vle_join_clean_withdummy)

[1] 295

[1] "code_module.x"       "code_presentation.x" "id_student"         
 [4] "date"                "sum_click"           "NA_dataplus"        
 [7] "NA_dualpane"         "NA_externalquiz"     "NA_folder"          
[10] "NA_forumng"          "NA_glossary"         "NA_homepage"        
[13] "NA_htmlactivity"     "NA_oucollaborate"    "NA_oucontent"       
[16] "NA_ouelluminate"     "NA_ouwiki"           "NA_page"            
[19] "NA_questionnaire"    "NA_quiz"             "NA_repeatactivity"  
[22] "NA_resource"         "NA_sharedsubpage"    "NA_subpage"         
[25] "NA_url"

> ### NExt we group by and summarise the interaction with vle

In [19]:
grouping_vle_join = vle_join_clean_withdummy %>% group_by(code_module.x,code_presentation.x,id_student) %>% summarise_at(c('sum_click', 'NA_dataplus', 'NA_dualpane', 
                                                                                                                  'NA_externalquiz', 'NA_folder' ,'NA_forumng'
                                                                                                                  ,'NA_glossary' ,'NA_homepage', 'NA_htmlactivity',
                                                                                                                  'NA_oucollaborate', 'NA_oucontent', 'NA_ouelluminate',
                                                                                                                  'NA_ouwiki','NA_page','NA_questionnaire','NA_quiz','NA_repeatactivity','NA_resource',
                                                                                                                  'NA_sharedsubpage','NA_subpage','NA_url'),sum, na.rm = TRUE)


> ### Next we created a unique key combining code_modeule, code_presentation and id_student

In [20]:
#making a unique key code_module.x code_presentation.x and id_student
key = paste(grouping_vle_join$code_module.x,grouping_vle_join$code_presentation.x,grouping_vle_join$id_student)


In [21]:
grouping_vle_join$key = key

In [22]:
nrow(grouping_vle_join)

[1] 29228

> ### Making the same unique key for joined student info and student registration table.

In [23]:
key= paste(std_join_info_reg$code_module.x,std_join_info_reg$code_presentation.x,std_join_info_reg$id_student)

In [24]:
std_join_info_reg$key = key

In [25]:
nrow(std_join_info_reg)

[1] 32734

> ### Joining joined std_join_info_reg with grouping_vle_join with our unique key with left join

In [26]:
#joining vle join table with std join table
std_join_vle = std_join_info_reg %>% left_join(grouping_vle_join, by = "key") 

In [27]:
nrow(std_join_vle)

[1] 32734

## Preparing assess_join to join the main table.
> ## Making dummy for assessment_type

In [28]:
#dummy for Assessment_type
tom = dummy(assess_join$assessment_type,sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"

In [29]:
assess_join = cbind(assess_join,tom)

In [30]:
head(assess_join[assess_join$NA_TMA ==1,])

id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,NA_CMA,NA_Exam,NA_TMA
1752,11391,18,0,78,AAA,2013J,TMA,19,10,0,0,1
1752,28400,22,0,70,AAA,2013J,TMA,19,10,0,0,1
1752,31604,17,0,72,AAA,2013J,TMA,19,10,0,0,1
1752,32885,26,0,69,AAA,2013J,TMA,19,10,0,0,1
1752,38053,19,0,79,AAA,2013J,TMA,19,10,0,0,1
1752,45462,20,0,70,AAA,2013J,TMA,19,10,0,0,1


In [31]:
nrow(assess_join)

[1] 173912

In [32]:
#dropping the date for the assessment since date has no impact on the scores
nrow(assess_join[!assess_join$NA_Exam ==1,])

[1] 168953

In [33]:
# making key for assess_join
key = paste(assess_join$code_module,assess_join$code_presentation,assess_join$id_student)

In [34]:
assess_join$key = key

In [35]:
# weighted assessment 

assess_join$weighted_TMA = ((assess_join$NA_TMA * assess_join$weight)/100)*assess_join$score
assess_join$weighted_Exam = ((assess_join$NA_Exam * assess_join$weight)/100)*assess_join$score
assess_join$weighted_CMA = ((assess_join$NA_CMA * assess_join$weight)/100)*assess_join$score

In [36]:
join_ = assess_join %>% group_by(key) %>% summarise(weight= sum(weight),CMA_count=sum(NA_CMA),Exam_count=sum(NA_Exam),TMA_count=sum(NA_TMA),
                                                    TMA_score = sum(weighted_TMA),Exam_score=sum(weighted_Exam), CMA_Score=sum(weighted_CMA))

In [37]:
nrow(assess_join)
#after grouping
nrow(join_)

[1] 173912

[1] 25843

In [38]:
# joining assessment_join_grouped with main_table
main_table = std_join_vle


In [39]:
#joining assessment table with main_table
main_table_join= main_table %>% left_join(join_, by = "key")

In [43]:
nrow(main_table_join)

[1] 32734

In [44]:
#Making key for Main table and course to join later
key1 = paste(courses$code_module,courses$code_presentation)
key2 = paste(main_table_join$code_module.x.x,main_table_join$code_presentation.x.x)

In [45]:
main_table_join$key= key2
courses$key = key1

In [46]:
finaltable_joined = main_table_join %>% left_join(courses, by = "key")

In [47]:
nrow(finaltable_joined)

[1] 32734

In [48]:
nrow(finaltable_joined[!complete.cases(finaltable_joined),])

[1] 8256

In [49]:
names(finaltable_joined)

[1] "code_module.x.x"            "code_presentation.x.x"     
 [3] "id_student.x"               "date_registration"         
 [5] "date_unregistration"        "X"                         
 [7] "gender"                     "region"                    
 [9] "highest_education"          "imd_band"                  
[11] "age_band"                   "num_of_prev_attempts"      
[13] "studied_credits"            "disability"                
[15] "final_result"               "key"                       
[17] "code_module.x.y"            "code_presentation.x.y"     
[19] "id_student.y"               "sum_click"                 
[21] "NA_dataplus"                "NA_dualpane"               
[23] "NA_externalquiz"            "NA_folder"                 
[25] "NA_forumng"                 "NA_glossary"               
[27] "NA_homepage"                "NA_htmlactivity"           
[29] "NA_oucollaborate"           "NA_oucontent"              
[31] "NA_ouelluminate"            "NA_ouwiki"                 
[33] "NA_page"                    "NA_questionnaire"          
[35] "NA_quiz"                    "NA_repeatactivity"         
[37] "NA_resource"                "NA_sharedsubpage"          
[39] "NA_subpage"                 "NA_url"                    
[41] "weight"                     "CMA_count"                 
[43] "Exam_count"                 "TMA_count"                 
[45] "TMA_score"                  "Exam_score"                
[47] "CMA_Score"                  "code_module"               
[49] "code_presentation"          "module_presentation_length"

In [50]:
#Deleting duplicate columns from final dataframe
finaltable_joined = finaltable_joined[,-which(names(finaltable_joined) %in% c("code_module.x.y","code_presentation.x.y",
                                                                              "code_module","code_presentation","id_student.y"))]

In [51]:
finaltable_joined$imd_band = is.character(finaltable_joined$imd_band)

In [52]:
finaltable_joined[which(finaltable_joined$imd_band =="10-20"),"imd_band"] = "10-20%"

In [53]:
finaltable_joined$key = as.factor(finaltable_joined$key)
lev = levels(finaltable_joined$key)

for (a in 1:length(lev)){
   finaltable_joined[is.na(finaltable_joined$date_registration) ,"date_registration"] = 
    mean(finaltable_joined[!is.na(finaltable_joined$date_registration) & finaltable_joined$key == lev[a],"date_registration"])
}

In [55]:
write.csv(finaltable_joined,"C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\final_data1.1.csv", row.names = FALSE)

In [56]:
head(finaltable_joined[complete.cases(finaltable_joined),which(names(finaltable_joined) %in% c("final_result","TMA_score","Exam_score",
                                                                                         "CMA_Score"))])

final_result,TMA_score,Exam_score,CMA_Score
Pass,82.4,0,0
Pass,65.4,0,0
Pass,76.3,0,0
Pass,72.5,0,0
Pass,71.2,0,0
Pass,79.0,0,0


In [57]:
nrow(finaltable_joined[!complete.cases(finaltable_joined),])

[1] 7261

In [58]:
names(finaltable_joined)

[1] "code_module.x.x"            "code_presentation.x.x"     
 [3] "id_student.x"               "date_registration"         
 [5] "date_unregistration"        "X"                         
 [7] "gender"                     "region"                    
 [9] "highest_education"          "imd_band"                  
[11] "age_band"                   "num_of_prev_attempts"      
[13] "studied_credits"            "disability"                
[15] "final_result"               "key"                       
[17] "sum_click"                  "NA_dataplus"               
[19] "NA_dualpane"                "NA_externalquiz"           
[21] "NA_folder"                  "NA_forumng"                
[23] "NA_glossary"                "NA_homepage"               
[25] "NA_htmlactivity"            "NA_oucollaborate"          
[27] "NA_oucontent"               "NA_ouelluminate"           
[29] "NA_ouwiki"                  "NA_page"                   
[31] "NA_questionnaire"           "NA_quiz"                   
[33] "NA_repeatactivity"          "NA_resource"               
[35] "NA_sharedsubpage"           "NA_subpage"                
[37] "NA_url"                     "weight"                    
[39] "CMA_count"                  "Exam_count"                
[41] "TMA_count"                  "TMA_score"                 
[43] "Exam_score"                 "CMA_Score"                 
[45] "module_presentation_length"

In [59]:
nrow(finaltable_joined)
finaltable_joined[is.na(finaltable_joined$sum_click),"sum_click"] = 0
finaltable_joined[is.na(finaltable_joined$NA_dataplus),"NA_dataplus"] = 0
finaltable_joined[is.na(finaltable_joined$NA_dualpane),"NA_dualpane"] = 0
finaltable_joined[is.na(finaltable_joined$NA_externalquiz),"NA_externalquiz"] = 0
finaltable_joined[is.na(finaltable_joined$NA_folder),"NA_folder"] = 0
finaltable_joined[is.na(finaltable_joined$NA_forumng),"NA_forumng"] = 0
finaltable_joined[is.na(finaltable_joined$NA_glossary),"NA_glossary"] = 0
finaltable_joined[is.na(finaltable_joined$NA_homepage),"NA_homepage"] = 0
finaltable_joined[is.na(finaltable_joined$NA_htmlactivity),"NA_htmlactivity"] = 0
finaltable_joined[is.na(finaltable_joined$NA_oucollaborate),"NA_oucollaborate"] = 0
finaltable_joined[is.na(finaltable_joined$NA_oucontent),"NA_oucontent"] = 0
finaltable_joined[is.na(finaltable_joined$NA_ouelluminate),"NA_ouelluminate"] = 0
finaltable_joined[is.na(finaltable_joined$NA_ouwiki),"NA_ouwiki"] = 0
finaltable_joined[is.na(finaltable_joined$NA_page),"NA_page"] = 0
finaltable_joined[is.na(finaltable_joined$NA_quiz),"NA_quiz"] = 0
finaltable_joined[is.na(finaltable_joined$NA_questionnaire),"NA_questionnaire"] = 0
finaltable_joined[is.na(finaltable_joined$NA_repeatactivity),"NA_repeatactivity"] = 0
finaltable_joined[is.na(finaltable_joined$NA_resource),"NA_resource"] = 0
finaltable_joined[is.na(finaltable_joined$NA_sharedsubpage),"NA_sharedsubpage"] = 0
finaltable_joined[is.na(finaltable_joined$NA_subpage),"NA_subpage"] = 0
finaltable_joined[is.na(finaltable_joined$NA_url),"NA_url"] = 0
finaltable_joined[is.na(finaltable_joined$weight),"weight"] = 0
finaltable_joined[is.na(finaltable_joined$CMA_count),"CMA_count"] = 0
finaltable_joined[is.na(finaltable_joined$Exam_count),"Exam_count"] = 0
finaltable_joined[is.na(finaltable_joined$TMA_count),"TMA_count"] = 0
finaltable_joined[is.na(finaltable_joined$TMA_score),"TMA_score"] = 0
finaltable_joined[is.na(finaltable_joined$Exam_score),"Exam_score"] = 0
finaltable_joined[is.na(finaltable_joined$CMA_Score),"CMA_Score"] = 0

[1] 32734

In [60]:
nrow(finaltable_joined[!complete.cases(finaltable_joined),])

[1] 0

In [61]:
sapply(finaltable_joined, function(x) sum(is.na(x)))

code_module.x.x      code_presentation.x.x 
                         0                          0 
              id_student.x          date_registration 
                         0                          0 
       date_unregistration                          X 
                         0                          0 
                    gender                     region 
                         0                          0 
         highest_education                   imd_band 
                         0                          0 
                  age_band       num_of_prev_attempts 
                         0                          0 
           studied_credits                 disability 
                         0                          0 
              final_result                        key 
                         0                          0 
                 sum_click                NA_dataplus 
                         0                          0 
               NA_dualpane            NA_externalquiz 
                         0                          0 
                 NA_folder                 NA_forumng 
                         0                          0 
               NA_glossary                NA_homepage 
                         0                          0 
           NA_htmlactivity           NA_oucollaborate 
                         0                          0 
              NA_oucontent            NA_ouelluminate 
                         0                          0 
                 NA_ouwiki                    NA_page 
                         0                          0 
          NA_questionnaire                    NA_quiz 
                         0                          0 
         NA_repeatactivity                NA_resource 
                         0                          0 
          NA_sharedsubpage                 NA_subpage 
                         0                          0 
                    NA_url                     weight 
                         0                          0 
                 CMA_count                 Exam_count 
                         0                          0 
                 TMA_count                  TMA_score 
                         0                          0 
                Exam_score                  CMA_Score 
                         0                          0 
module_presentation_length 
                         0

In [62]:
write.csv(finaltable_joined,"C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\final_data2.csv", row.names = FALSE)